In [5]:
%load_ext wurlitzer
#^^^ used to capture C/C++ output to be displayed in the notebook

import time
from klampt import *
from IPython.display import clear_output
import ipywidgets as widgets
import ipyklampt

#get the HOME/Klampt base directory
import os
KLAMPT_DIR = os.environ['HOME'] + '/Klampt'

world = WorldModel()
if world.loadFile(KLAMPT_DIR + "/data/tx90scenario0.xml"):
    sim = Simulator(world)
    kvis = ipyklampt.KlamptWidget(world)
    playback_widget = ipyklampt.Playback(kvis)
    
    #If you'd like to show the print output from loading the file, comment out this line
    clear_output()
    display(kvis)
    display(playback_widget)
    
    #NOTE: if you are going to add/modify items to the world in the same cell that it is created, you will
    #need to place all of those calls in a begin_rpc/end_rpc block
    kvis.begin_rpc()
    kvis.add_text("HUD1",1,1,"0")
    ghost = kvis.add_ghost()
    kvis.set_color(ghost,1,0,0,0.5)
    kvis.end_rpc()
else:
    print "There was a problem loading the world file"
    
#Controls:
#left mouse click to rotate the view
#right click or ctrl+click to pan the view
#mouse wheel or shift+click to zoom the view

KlamptWidget(scene={u'object': {u'matrix': [1, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 1], u'uuid': u'357ee…

Playback(children=(HBox(children=(Button(description=u'Play', icon=u'play', style=ButtonStyle(), tooltip=u'Sta…

In [6]:
import random
from klampt.plan.cspace import *
from klampt.plan.robotcspace import RobotCSpace
from klampt.model.collide import WorldCollider
from klampt.model.trajectory import *

#first time the plan-to is called
next_move_time = 1.0
#if this is too large, Jupyter will complain...
planning_time_limit = 2.0

robot = world.robot(0)

def plan_to(qa,qb):
    collider = WorldCollider(world)
    cspace = RobotCSpace(robot,collider)
    cspace.eps = 1e-2
    optimizing = True
    MotionPlan.setOptions(type="sbl",connectionThreshold=2,perturbationRadius=2,shortcut=True)
    planner = MotionPlan(cspace)
    planner.setEndpoints(qa,qb)
    t0 = time.time()
    foundTime = None
    path = None
    kvis.add_text("HUD2",1,5,"Planning...")
    while time.time() - t0 < planning_time_limit:
        planner.planMore(1)
        if foundTime == None:
            path = planner.getPath()
            if path != None and len(path) > 0:
                foundTime = time.time()
                kvis.add_text("HUD2",1,5,"Found first path in %gs"%(foundTime-t0,))
                if not optimizing:
                    break
    if foundTime != None:
        if optimizing:
            kvis.add_text("HUD2",1,5,"Optimized for another %gs"%(time.time()-foundTime,))
        path = planner.getPath()
    else:
        path = None
    return path


def control_loop(t,controller):
    global next_move_time
    kvis.add_text("HUD1",1,1,str(t))
    if t+0.02 >= next_move_time:
        print "Planning on next time step for %fs..."%(planning_time_limit,)
    if t >= next_move_time:
        qmin,qmax = robot.getJointLimits()
        q = [random.uniform(a,b) for (a,b) in zip(qmin,qmax)]
        print "Calling plan to...",q
        kvis.set_color(ghost,1,1,0,0.5)
        kvis.set_ghost_config(q)
        try:
            path = plan_to(controller.getCommandedConfig(),q)
        except Exception as e:
            print e
            path = None
        if path != None:
            kvis.set_color(ghost,0,1,0,0.5)
            for i,q in enumerate(path):
                if i == 1:
                    controller.addMilestoneLinear(q)
                elif i >= 2:
                    controller.addMilestoneLinear(q)
        else:
            kvis.set_color(ghost,1,0,0,0.5)
        #controller.setMilestone(q)
        next_move_time += 5.0
    pass


In [7]:
dt = 0.02

def do_reset():
    global next_move_time
    sim.reset()
    sim.updateWorld()
    next_move_time = 2.0

def do_advance():
    global world,sim
    if world.numRobots() > 0:
        control_loop(sim.getTime(),sim.controller(0))  #call code in the above cell
    #print "Simulating...",dt
    sim.simulate(dt)
    sim.updateWorld()
    #print "Done."

#this binds the playback widget buttons
playback_widget.advance = do_advance
playback_widget.reset = do_reset
playback_widget.quiet = False